In [93]:
### SCRIPT PARA LIMPIAR LOS GEOJSONS
# Quitar la tercera coordenada, poner latitud y longitud en orden correcta(inversa)...

import json

def clean_geojson(gjfile):
    with open(gjfile, 'r') as file:
        gj_data = json.load(file)
        lista_coord = gj_data['features']['geometry']['coordinates'][0]
        for i in range(len(lista_coord)):
            if len(lista_coord[i]) == 3:
                lista_coord[i] = lista_coord[i][1::-1]
    with open(gjfile, 'w') as file:
        json.dump(gj_data, file, indent=2)

lista_distritos = ['bailen-miraflores', 'campanillas', 'carretera-de-cadiz',
                  'centro', 'churriana', 'ciudad-jardin', 'cruz-de-humilladero', 'este', 'martiricos-la-roca', 
                  'puerto-de-la-torre', 'teatinos'
                  ]

for distrito in lista_distritos:
    distrito_geojson_path = './GeoJSON Distritos Malaga/geoJson_Malaga_' + distrito + '.geojson'
    clean_geojson(distrito_geojson_path)


In [127]:
import folium
import json
from shapely.geometry import Point, shape
import pandas as pd


lista_distritos = ['bailen-miraflores', 'campanillas', 'carretera-de-cadiz',
                'centro', 'churriana', 'ciudad-jardin', 'cruz-de-humilladero', 'martiricos-la-roca', 
                'puerto-de-la-torre', 'teatinos', 'este'
                ]

lista_distrito_apt = []

data_airdna = pd.read_csv('./DB_airDNA.csv', index_col=0)

for index, row in data_airdna.iterrows():
    apt_coord = row['lat'], row['long']

    apt_point = Point(apt_coord)

    distrito_apt = None

    # mapa = folium.Map(location=[36.720892, -4.420732], zoom_start=12, width=1000, height=1200)
    # folium.Marker(apt_coord, popup='Apartment').add_to(mapa)
    # lista_colores = ['red', 'darkred', 'fuchsia', 'orange', 'blue', 'green', 'pink', 'lightgreen', 'purple', 'yellow', 'teal'] 

    for i in range(len(lista_distritos)):

        distrito_geojson_path = './GeoJSON Distritos Malaga/geoJson_Malaga_' + lista_distritos[i] + '.geojson'

        with open(distrito_geojson_path, 'r') as geojson_file:
            geojson_data = json.load(geojson_file)    
            distrito_poly = shape(geojson_data['features']['geometry'])
            # folium.Polygon(locations=geojson_data['features']['geometry']['coordinates'][0], color=lista_colores[i], fill=True, fill_color=lista_colores[i], fill_opacity=0.5).add_to(mapa)
        
        if apt_point.within(distrito_poly):
            distrito_apt = lista_distritos[i].capitalize()        
            # print('El apartamento se encuentra en el distrito: ' + lista_distritos[i].capitalize())
    
    lista_distrito_apt.append(distrito_apt)

data_airdna['Distrito']  = lista_distrito_apt
data_airdna




,property_id,airbnb_property_id,vrbo_property_id,listing_type,bedrooms,bathrooms,accommodates,rating,reviews,title,revenue_ltm,revenue_potential_ltm,occupancy_rate_ltm,average_daily_rate_ltm,days_available_ltm,market_id,market_name,lat,long,Distrito
0,abnb_10802138,10802138.0,NaN,private_room,1,1.0,1,4.6,825,Room next to Airport A/C,6812.86,8455.55,75.26,31.54,287,30838,Málaga,36.703900,-4.449810,Cruz-de-humilladero
1,abnb_30141259,30141259.0,NaN,entire_place,1,1.0,4,4.8,810,Apartamento Centro Málaga+Garaje Privado Gratis,27010.26,28506.77,84.20,92.18,348,30838,Málaga,36.722300,-4.427160,Centro
2,abnb_2343063,2343063.0,NaN,entire_place,0,1.0,2,4.8,803,Nice Flat in the center VFT/MA/00202,17090.18,17684.94,100.00,48.69,351,30838,Málaga,36.726090,-4.419700,Centro
3,abnb_2336145,2336145.0,NaN,entire_place,1,1.0,2,4.8,720,Lovely apartment in the center VFT/MA/00204,17183.36,21824.55,100.00,59.26,290,30838,Málaga,36.724940,-4.420030,Centro
4,abnb_13315721,13315721.0,NaN,entire_place,0,1.0,2,5.0,588,"Relax, culture and beach in the heart of the c...",22396.48,23691.99,99.42,65.68,343,30838,Málaga,36.721340,-4.418110,Centro
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2317,abnb_53107615,53107615.0,NaN,entire_place,1,1.0,2,4.5,68,Acogedor apartamento junto al museo Picasso.,9415.77,16421.41,83.42,58.48,193,30838,Málaga,36.721977,-4.418969,Centro
2318,abnb_47189457,47189457.0,10818662ha,entire_place,1,1.0,4,4.6,68,La Mar Salada - MálagadeVacaciones,33577.36,39161.41,82.87,126.23,321,30838,Málaga,36.718110,-4.409590,Centro
2319,abnb_19100716,19100716.0,8966772ha,entire_place,1,1.0,4,4.3,68,Sw Calle Capitan-Charming Duplex,21550.47,25030.45,82.62,85.51,305,30838,Málaga,36.721360,-4.419990,Centro
2320,abnb_43289530,43289530.0,NaN,private_room,1,1.0,2,4.5,68,Spacious room with private bathroom,20667.91,20667.91,82.19,68.90,365,30838,Málaga,36.729310,-4.411370,Centro


In [129]:
data_airdna.to_csv('./DB_airDNA_completo.csv')